In [ ]:
# !pip install pandas
# !pip install openai
# !pip install python-dotenv
import pandas as pd
import openai
from dotenv import load_dotenv
import os
from tqdm import tqdm
import multiprocessing
import shutil

In [ ]:
load_dotenv()

In [ ]:
openai.api_key = "Your_OpenAI_Key"

# Copying the data to the ../data/generated folder using python
shutil.copy('source_path', 'target_path')


# Creating a global lock for file writing
lock = multiprocessing.Lock()




def get_response(system_prompt, user_prompt):
    """Get the response from the OpenAI API."""
    response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-16k",
            # model="gpt-4",
              messages=[{"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_prompt},
                        ])

    message = response["choices"][0]["message"]["content"]
    return message


def generate(judgement_text):
    """Generate the response from the OpenAI API."""

    # Give general instructions in the system prompt
    system_prompt = """Your task is to generate nuanced, perspective-based summaries for given legal judgment texts from Indian Supreme Court cases. A 'perspective-based summary' aims to provide a multi-faceted understanding of the judgment. It should include a critical analysis of legal reasoning, implications for future cases, the impact on the parties involved, and broader emotional, social, or political implications. This goes beyond simply summarizing the judgment's facts and findings."""

    # Give more specific instructions in the user prompt and also provide the judgement text
    user_prompt = """Consider the following Indian Supreme Court case judgment text: Generate a perspective-based summary for the above judgment text in the perspective of:

1. The Initiator (e.g., Petitioner / Appellant / Prosecution):
Your summary should cover these areas:
-Legal Strengths: Identify legal precedents, statutes, or rules that the judgment confirms or establishes in favor of the initiator.
-Evidentiary Gains: Discuss how the judgment impacts the admissibility or weight of key evidence presented by the initiator.
-Judicial Reasoning: Comment on the judge’s reasoning that aligns with or strengthens the initiator’s arguments.
Additionally, feel free to consider other significant factors not listed here that impact the initiator’s case.

2. The Respondent (e.g., Defendant / Accused / Opposition Party):
Your summary should cover these areas:
-Legal Strengths: Identify legal precedents, statutes, or rules that the judgment confirms or establishes in favor of the respondent.
-Evidentiary Gains: Discuss elements in the judgment that question the admissibility or reliability of the initiator’s evidence.
-Judicial Reasoning: Highlight portions of the judge’s reasoning that are favorable to the respondent or offer grounds for an appeal.
Also, you should incorporate any other crucial aspects not explicitly mentioned that could affect the respondent’s perspective.

Note:

The summaries should offer nuanced insights specific to each role and not merely serve as mirror opposites.

Keep the tone academic and the perspective in the third person.
Format your response as a single JSON object with two properties: initiator and respondent. Each should encapsulate the relevant perspectives in a detailed, coherent manner and should be at least 150 words long each.
{{
  "initiator": "From the viewpoint of the initiator, the judgment affirms that interest accrued—even if not realized in cash—must be included in the assessee's net wealth as per the Wealth Tax Act. This reinforces the legal principle that statutory obligations cannot be avoided based on the method of accounting, thereby strengthening the prosecution’s argument. The court's acceptance of accrued interest as valid evidence adds substantial weight to financial records reflecting potential but unrealized income. Judicial reasoning favored a strict interpretation of ‘net wealth,’ aligning with the initiator’s case, and emphasized the relevance of economic value on the valuation date over actual receipt. This decision enhances the initiator’s ability to pursue hidden or deferred assets under tax obligations.", 
  "respondent": "From the respondent’s perspective, the judgment acknowledges conflicting interpretations across different High Courts, which opens avenues for challenging the Supreme Court’s stance. Although the judgment does not dismiss the evidence submitted by the initiator, it implicitly raises the issue of how accounting systems influence tax liabilities. The respondent can leverage the observed judicial acknowledgment of legal ambiguities as a basis for appeal. The defense may argue that applying the Wealth Tax Act without considering the assessee’s cash-based accounting could be unfair and lacking nuance. This case thus serves as a potential basis for arguing for harmonization of judicial interpretations in future disputes involving accrual versus cash accounting principles."
}}
    """.format(judgement_text)

    return get_response(system_prompt, user_prompt)


def generate_and_save(args):
    index, judgement, filepath = args
    try:
        new_text = generate(judgement)
        # Save the result immediately
        with lock:
            df = pd.read_csv(filepath)
            df.at[index, 'GeneratedText'] = new_text
            df.to_csv(filepath, index=False)
        return (index, new_text)
    except Exception as e:
        return (index, None)


def generate_and_save_parallelly(filepath):

    try:
        # Loading the CSV into a DataFrame
        df = pd.read_csv(filepath)

        if 'Target_Column' not in df.columns:
            df['Target_Column'] = None

        to_generate = [(index, row['Text'], filepath) for index, row in df.iterrows() if pd.isnull(row['Target_Column']) or row['Target_Column'] == ""]

        with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
            list(tqdm(pool.imap(generate_and_save, to_generate), total=len(to_generate), desc="Generating text"))

        # Checking if all rows have been generated
        df = pd.read_csv(filepath)
        if df['Target_Column'].isnull().sum() == 0:
            print("All rows have been generated.")

    except KeyboardInterrupt:
        print("Process interrupted by user. Exiting...")


if __name__ == "__main__":
    generate_and_save_parallelly("target_path")